In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.endpoints import leaguegamefinder

import matplotlib.pyplot
import pandas as pd
import time

In [13]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from datetime import datetime
#need to get list of all teams to avoid g league playoff games
nba_teams = teams.get_teams()
nba_team_ids = {team['id'] for team in nba_teams}
nba_team_abbreviations = {team['abbreviation'] for team in nba_teams}
# Set the season type to 'Playoffs'
season_type = 'Playoffs'

# Get the game logs for the playoffs
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable=season_type)
games = gamefinder.get_data_frames()[0]

# Filter the game logs to only include games from 2020 onwards
games['GAME_DATE'] = games['GAME_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
games_since_2000 = games[(games['GAME_DATE'].dt.year >= 2000)] #& (games['GAME_DATE'].dt.year <= 2020)]

games_since_2000['OPPONENT_ABBREVIATION'] = games_since_2000['MATCHUP'].apply(
    lambda x: x.split()[-1]
)

games_since_2000 = games_since_2000[
    (games_since_2000['TEAM_ABBREVIATION'].isin(nba_team_abbreviations)) & 
    (games_since_2000['OPPONENT_ABBREVIATION'].isin(nba_team_abbreviations))
]

games_since_2000

C:\Users\syedo\AppData\Local\Temp\ipykernel_20612\2153906455.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2000['OPPONENT_ABBREVIATION'] = games_since_2000['MATCHUP'].apply(


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,OPPONENT_ABBREVIATION
0,42023,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,L,240,88,...,7,28,35,18,4,4,13,20,-18.0,BOS
1,42023,1610612738,BOS,Boston Celtics,0042300405,2024-06-17,BOS vs. DAL,W,240,106,...,15,36,51,25,9,2,7,15,18.0,DAL
2,42023,1610612742,DAL,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,W,240,122,...,13,39,52,21,7,2,8,17,38.0,BOS
3,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,239,84,...,4,27,31,18,2,5,13,19,-38.0,DAL
4,42023,1610612742,DAL,Dallas Mavericks,0042300403,2024-06-12,DAL vs. BOS,L,240,99,...,7,36,43,15,5,1,8,17,-7.0,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,41999,1610612758,SAC,Sacramento Kings,0049900007,2000-04-23,SAC @ LAL,L,235,107,...,15,22,37,18,7,3,12,19,-10.0,LAL
5544,41999,1610612759,SAS,San Antonio Spurs,0049900002,2000-04-22,SAS vs. PHX,L,236,70,...,9,41,50,15,7,9,18,19,-2.0,PHX
5546,41999,1610612765,DET,Detroit Pistons,0049900001,2000-04-22,DET @ MIA,L,234,85,...,9,28,37,18,4,1,17,26,-10.0,MIA
5549,41999,1610612756,PHX,Phoenix Suns,0049900002,2000-04-22,PHX @ SAS,W,235,72,...,10,37,47,20,10,10,14,20,2.0,SAS


In [17]:


def get_games(year):
    season = f"{year}-{str(year+1)[2:]}"

    games = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable='Playoffs').get_data_frames()[0]
    return games


In [15]:
#function to determine the winner of each series since nba_api doesn't have implementation for that

def series_winner(game):
    series_groups = games.groupby(['SEASON_ID' , 'MATCHUP'])
    series_winners = []

    for name, group in series_groups:
        # Count wins for each team
        wins = group['TEAM_ID'].value_counts()
        # Determine the winner (team with more wins)
        winner = wins.idxmax()
        # Append series info and winner
        series_winners.append({'Season': name[0], 'Matchup': name[1], 'Winner': winner})

    return pd.DataFrame(series_winners)




In [21]:
all_series_winners = pd.DataFrame()

for year in range(2000, 2023):  # Update the end year as needed
    games = get_games(year)
    series_winners = series_winner(games_since_2000)
    all_series_winners = pd.concat([all_series_winners, series_winners], ignore_index=True)

In [22]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
team_id_name_map = {team['id']: team['full_name'] for team in nba_teams}

all_series_winners['Winner'] = all_series_winners['Winner'].map(team_id_name_map)

In [25]:
all_series_winners

,Season,Matchup,Winner
0,42000,CHH @ MIA,Charlotte Hornets
1,42000,CHH @ MIL,Charlotte Hornets
2,42000,CHH vs. MIA,Charlotte Hornets
3,42000,CHH vs. MIL,Charlotte Hornets
4,42000,DAL @ SAS,Dallas Mavericks
...,...,...,...
1753,42022,SLC vs. SXF,NaN
1754,42022,STO vs. SXF,NaN
1755,42022,SXF @ SLC,NaN
1756,42022,SXF @ STO,NaN
